### Distilling the Web Assignment


####  Christina Chappell
#### February 20, 2017

Site: http://students.washington.edu/chappchr/movies_wa.html

##### Importing Packages
This will do all the importing to let me do my work.

In [1]:
import urllib2
import lxml
from lxml import html
import cssselect


In [2]:
import json
import requests


In [3]:
import pandas
import folium

In [4]:
import random

##### Scraping the website
This grabs the correct website we want to scrape which is : http://www.onlyinyourstate.com/washington/washington-movies/

I used this website because I am interested in where movies are filmed, and specifically where movies were filmed in Washington. This website talks about 8 movies that were filmed in WA.

estimated time: 3 hours

In [10]:
url = "http://www.onlyinyourstate.com/washington/washington-movies/"
req = urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}) 
con = urllib2.urlopen( req )
doc_text = con.read()
doc = lxml.html.fromstring(doc_text)
doc.make_links_absolute(url)

##### Scraping for movie description
Here we created a list of the titles we scraped from the chosen website. The selector we chose here was the root for all of the individual articles. From there we created another selector to grab the actual data we needed for our lists.

In [11]:
description_list = []
for row in doc.cssselect('#gallery-1 figure'):
    single_row = row.cssselect("div[class=' gallery-description ']")
    if len(single_row) != 0:
        event_title = single_row.pop().text
        description_list.append(event_title)
description_list[1] = "Monroe is a city in Snohomish County" 
# this description would return "Republic of Namibia" which I think is the default value when no place is detected.
# So I hard coded the location that was within that movie description to clean the data.

##### Scraping for movie name
This creates the list of dates for the same website as above. The logic for our selectors is the same as above, just changed to fit the dates.

In [12]:
movie_list = []
for row in doc.cssselect('#gallery-1 figure'):
    single_row = row.cssselect("figcaption[class=' gallery-caption ']") 
    if len(single_row) != 0:
        event_title = single_row.pop().text
        movie_list.append(event_title)


##### Geoparser
This creates an empty json list to hold the geo references from the api that will be done down below

estimated time: 4 hours

In [13]:
json_list = [0]*8

Here I used the documentation from the geoparser.io to create the correct information needed to complete this assignment. This finds the locations of all the places that have been used to film based on the desciptions above.

In [51]:
url = 'https://geoparser.io/api/geoparser'
headers = {'Authorization': 'apiKey 94861936067943054'}
counter = 0
for row in description_list:
    data = {'inputText': str(description_list[counter])}
    response = requests.post(url, headers= headers, data= data)
    json_file = json.dumps(response.json(), indent=2)
    json_as_dictionary = json.loads(json_file)
    json_list[counter] = json_as_dictionary
    counter += 1


##### Color generator
I used this code to make a random color generator http://stackoverflow.com/questions/13998901/generating-a-random-hex-color-in-python 
this creates a list for unique random colors in hex format for the number of movies that are in the movie_list.

In [52]:
#make marker colors with dictionary 
color_list = []
count_mov = 0
for row in movie_list:
    r = lambda: random.randint(0,255)
    color = ('#%02X%02X%02X' % (r(),r(),r()))
    color_list.append(color)
    count_mov += 1


##### Complete lists for each place
This creates the lists that will be used for the geodataframe. The lists consist of one point for each place that a movie is filmed in. Some movies have multiple locations so there are multiple rows for a single movie with different point 

estimated time: 5 hours

In [57]:
counter = 0
total_dict = {}
#new lists for the dataframe
movies = []
description = []
location_list = []
new_color = []
#finding the coordinates for each point and creating the final list all of the same length
for movie in movie_list:
    movie_dict = json_list[counter]
    for location in movie_dict['features']:
        movie_shoot =  location['geometry']['coordinates']
        print movie_shoot
        movies.append(movie_list[counter]) 
        description.append(description_list[counter]) 
        location_list.append(movie_shoot)
        new_color.append(color_list[counter])
    counter +=1


[48.11815, -123.43074]
[47.70204, -122.62098]
[39.76, -98.5]
[48.04602, -121.72218]
[47.60621, -122.33207]
[47.50012, -120.50147]
[47.60621, -122.33207]
[48.5126, -122.61267]
[48.86381, -117.37247]
[47.50012, -120.50147]
[48.24121, -122.37071]
[44.00013, -120.50139]
[48.2926, -122.67628]
[47.50012, -120.50147]
[37.25022, -119.75126]
[-25.0, 122.0]
[48.75955, -122.48822]
[47.60621, -122.33207]
[48.11704, -122.76045]
[47.50012, -120.50147]
[47.60621, -122.33207]
[47.25288, -122.44429]


This sometimes works to reverse the lat/long data. sometimes it doesn't... I don't really understand. It works once, but if I keep running my code it re-reverses it back to the wrong order and my map won't work.


In [58]:
#counter_test = 0
#for row in location_list:
    #location_list[counter_test].reverse()
    #print location_list[counter]
    #counter_test += 1
    

##### Create the GeoDataFrame
This creates the dataframe for all the data I scraped and the correct geocodes for the places in the movie descriptions. This also creates the csv file to easily view the resulting dataframe.

In [59]:
dataf = {'Movie' : movies, 'Description' : description, 'Location' : location_list, 'Color' : new_color}

pandas_df = pandas.DataFrame(dataf)
#make the geodataframe into a csv file.
pandas_df.to_csv("movie_file1.csv") 


In [60]:
pandas_df.head()

,Color,Description,Location,Movie
0,#D00212,"Based on a popular Tom Clancy novel, this movi...","[48.11815, -123.43074]","""The Hunt For Red October"", 1990"
1,#D00212,"Based on a popular Tom Clancy novel, this movi...","[47.70204, -122.62098]","""The Hunt For Red October"", 1990"
2,#D00212,"Based on a popular Tom Clancy novel, this movi...","[39.76, -98.5]","""The Hunt For Red October"", 1990"
3,#C71DE4,Monroe is a city in Snohomish County,"[48.04602, -121.72218]","""The Butterfly Effect"", 2004"
4,#B901EF,"If the title doesn't ring a bell, chances are ...","[47.60621, -122.33207]","""Say Anything..."", 1989"


##### Create the Map
Here I am setting up the folium map with the coordinates and zoom that I want. I chose this zoom because the data is showing where films were shot in Washington State. The tile I chose was clean and organized looking. I used circles because the hex code I generated would not work with the default icon. Each movie has a different color so it is easier to see where which movies filmed in Washington State

estimated time: 4 hours

In [76]:
#initialize the map
map_center_lat = 47.6
map_center_long = -120.5
map_zoom = 7
my_map = folium.Map(location=[map_center_lat, map_center_long], zoom_start= map_zoom, tiles="cartodbpositron")
#making the individual points
count = 0
for row in location_list:
    one_point = location_list[count]
    folium.CircleMarker([one_point[0],one_point[1]], 
                        radius=8000,
                        popup= movies[count], 
                        color=new_color[count],
                       fill_color=new_color[count]).add_to(my_map)
    count += 1
#saves as an html to view on the web    
my_map.save('movies_wa.html')
my_map

##### Making the Shapfile

Here I attempted to make a shapefile with my data. I was able to put the lat/long of all my points into a shapefile but without the data that goes with it. There are no movie names or descritions for the movies, just the points. There is also a spatial reference issue. I tried finding a way to make my folium map into a shapefile but was unsuccessfull, and the datafram.to_file was not working as well.

In [62]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy


In [6]:
arcpy.env.workspace = "U:\\Geography\\GEOG458" 

new_shapefile = arcpy.CreateFeatureclass_management("U:\\Geography\\GEOG458", 'movie_location1.shp', "POINT")

In [75]:
table_shp = []
pt = arcpy.Point()
for p in location_list:
    pt.X = p[1]
    pt.Y = p[0]
    table_shp.append(arcpy.PointGeometry(pt))
    
#creates a shapefile with no spatial reference...
arcpy.CopyFeatures_management(table_shp, r"U:\\Geography\\GEOG458\\test_shape.shp")


<Result 'U:\\\\Geography\\\\GEOG458\\test_shape.shp'>